<a href="https://colab.research.google.com/github/kookeej/DILAB/blob/main/7.1-7.7/lab05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lab05 - Logistic Regression
===


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
torch.manual_seed(1)

In [3]:
# Training Data
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]

In [4]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [5]:
print(x_train.shape)
print(y_train.shape)

torch.Size([6, 2])
torch.Size([6, 1])


# 1. Computing Hypothesis

* logistic regression의 hypothesis 식은 아래와 같다.    

![](https://render.githubusercontent.com/render/math?math=H%28X%29%20%3D%20%5Cfrac%7B1%7D%7B1%2Be%5E%7B-W%5ET%20X%7D%7D&mode=display)


```exponential function```같은 경우, ```torch.exp()```를 사용하여 나타낼 수 있다.

In [6]:
print('e^1 equals: ', torch.exp(torch.FloatTensor([1])))

e^1 equals:  tensor([2.7183])


또한, 이 exp()를 사용하여 hypothesis를 간편하게 나타낼 수 있다.

In [7]:
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

hypothesis = 1 / (1 + torch.exp(-(x_train.matmul(W) + b)))  #torch.matmul(x_train, W)

In [8]:
print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)
torch.Size([6, 1])


* 또는 우리는 ```torch.exp()``` 대신에 ```torch.sigmoid()```를 사용해서 식을 나타낼 수 있다.
```
hypothesis = torch.sigmoid(x_train.matmul(W) + b)
```
위 코드가 더 간결하다.

# 2. Computing Cost function
logistic regression의 cost function은 아래와 같다.     
![](https://render.githubusercontent.com/render/math?math=cost%28W%29%20%3D%20-%5Cfrac%7B1%7D%7Bm%7D%20%5Csum%20y%20%5Clog%5Cleft%28H%28x%29%5Cright%29%20%2B%20%281-y%29%20%5Cleft%28%20%5Clog%281-H%28x%29%20%5Cright%29&mode=display)

* 먼저 ```hypothesis```와 ```y_train```사이의 차이를 계산해보자.

In [9]:
print(hypothesis)
print(y_train)
print(y_train - hypothesis)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.]])
tensor([[-0.5000],
        [-0.5000],
        [-0.5000],
        [ 0.5000],
        [ 0.5000],
        [ 0.5000]], grad_fn=<SubBackward0>)


* 전체 data에 대해 loss를 계산해보면 아래와 같다.

In [10]:
losses = -(y_train * torch.log(hypothesis) + (1 - y_train) * torch.log(1 - hypothesis))
print(losses)

tensor([[0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931]], grad_fn=<NegBackward>)


In [11]:
# cost는 losses의 평균이다.
cost = losses.mean()
print(cost)

tensor(0.6931, grad_fn=<MeanBackward0>)


하지만 우리가 지금까지 적은 이 수식의 코드들을 한 번에 해결할 수 있다. 바로 ```F.binary_cross_entropy```를 사용하면 된다.
```
F.binary_cross_entropy(hypothesis, y_train)
```

In [12]:
bce = F.binary_cross_entropy(hypothesis, y_train)
print(bce)

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward>)


전체 코드를 살펴보자

In [13]:
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W, b], lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    hypothesis = torch.sigmoid(x_train.matmul(W) + b)
    cost = F.binary_cross_entropy(hypothesis, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost:{:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost:0.693147
Epoch  100/1000 Cost:0.134722
Epoch  200/1000 Cost:0.080643
Epoch  300/1000 Cost:0.057900
Epoch  400/1000 Cost:0.045300
Epoch  500/1000 Cost:0.037261
Epoch  600/1000 Cost:0.031672
Epoch  700/1000 Cost:0.027556
Epoch  800/1000 Cost:0.024394
Epoch  900/1000 Cost:0.021888
Epoch 1000/1000 Cost:0.019852


# 3. 성능 평가
우리의 model 학습을 끝냈으니 우리의 모델이 얼마나 잘 작동하는지 확인해보자.
```
hypothesis = torch.sigmoid(x_test.matmul(W) + b)
```

In [14]:
hypothesis = torch.sigmoid(x_train.matmul(W) + b)
prediction = hypothesis >= torch.FloatTensor([0.5])
print(prediction[:5])

tensor([[False],
        [False],
        [False],
        [ True],
        [ True]])


In [15]:
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(self.linear(x))

In [16]:
model = BinaryClassifier()

In [17]:
optimizer = optim.SGD(model.parameters(), lr=1)
nb_epochs = 100
for epoch in range(nb_epochs + 1):
    hypothesis = model(x_train)

    cost = F.binary_cross_entropy(hypothesis, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 10 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_train
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}'.format(
            epoch, nb_epochs, cost.item(), accuracy * 100,
        ))

Epoch    0/100 Cost: 0.539713 Accuracy 83.33
Epoch   10/100 Cost: 0.614853 Accuracy 66.67
Epoch   20/100 Cost: 0.441875 Accuracy 66.67
Epoch   30/100 Cost: 0.373145 Accuracy 83.33
Epoch   40/100 Cost: 0.316358 Accuracy 83.33
Epoch   50/100 Cost: 0.266094 Accuracy 83.33
Epoch   60/100 Cost: 0.220498 Accuracy 100.00
Epoch   70/100 Cost: 0.182095 Accuracy 100.00
Epoch   80/100 Cost: 0.157299 Accuracy 100.00
Epoch   90/100 Cost: 0.144091 Accuracy 100.00
Epoch  100/100 Cost: 0.134272 Accuracy 100.00
